In [1]:
"""
CardIA-Net: An Explainable Deep Learning Model for MI Detection with ECG Lead Optimization.

This module provides a reusable PyTorch implementation of the CardIA-Net model:
- Multi-scale 1D Inception modules with residual shortcut
- Channel-preserving Inception block (stack of Inception modules + residual add)
- Global average pooling head
- Lightweight vector attention over the pooled features
- Linear classifier

Typical input shape: (batch_size, num_leads, signal_length)

Preprint of this study: 
Bulbul, Abdullah Al-Mamun and Awal, Md Abdul and Aloteibi, Saad and Moni, Mohammad Ali, 
Cardia-Net: An Explainable Deep Learning Model for Mi Detection with Ecg Lead Optimization. 
Available at SSRN: https://ssrn.com/abstract=5387266 or http://dx.doi.org/10.2139/ssrn.5387266

PyTorch >= 1.10

"""


#######################################################################################
## CardIA-Net Model - TEST ONLY (load params + test set)
#######################################################################################

# Imports (kept minimal, no changes to architecture code)
import sys
import platform
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
import h5py
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import confusion_matrix, classification_report

# Reproducibility 
import random
import numpy as np
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Device setup
has_gpu = torch.cuda.is_available()
has_mps = torch.backends.mps.is_built()
device_str = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Python Platform: {platform.platform()}")
print(f"PyTorch Version: {torch.__version__}")
print(f"Using device: {device_str}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

########### Match your naming exactly ###########
index_combination = 31   
mod_weight = f'CardIA_Net_Trained_parameters.pth'  # load from working directory
PTB_CSN_test = f'PTB_CSN_test_set.h5'  # load from working directory
print("Expecting weight file:", mod_weight)
print("Expecting test set   :", PTB_CSN_test)



num_epochs = 1000      # Number of training epochs
batch_size = 32        # Batch Size
num_incep_block = 6    # Number of Inception Blocks
Num_CH = 4             # Number of input ECG leads
lr = 0.0001            # Learning rate
sig_len = 300          # Number of data-points in each ECG segment @500Hz



#######################################################################################
# Load the test dataset from the directory
#######################################################################################
with h5py.File(PTB_CSN_test, 'r') as hf:
    test_signals = hf['test_signals'][:]
    test_labels = hf['test_labels'][:]

# Ensure shapes align
test_labels = test_labels.reshape(-1)
print("Loaded test_signals shape:", test_signals.shape)
print("Loaded test_labels shape :", test_labels.shape)

# Determine channels from the saved test set
CH_No = test_signals.shape[1]
print("Inferred CH_No from test set:", CH_No)

# Dataloader (same helper behavior)
def create_dataloader(signals, labels, batch_size=batch_size, shuffle=False):
    signals = torch.tensor(signals, dtype=torch.float32)
    labels = torch.tensor(labels, dtype=torch.int64)
    dataset = TensorDataset(signals, labels)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=0)

test_loader = create_dataloader(test_signals, test_labels, batch_size=batch_size, shuffle=False)
print("Test DataLoader ready.")

#######################################################################################
## Define the Attention Mechanism Module 
#######################################################################################
    
# Define the attention mechanism module
class Attention(nn.Module):
    def __init__(self, input_dim):
        super(Attention, self).__init__()
        
        # Linear layer to compute attention scores, keeping the input dimension
        self.attention_weights = nn.Linear(input_dim, input_dim, bias=False)
        
        # Bias term for the addition step (b)
        self.bias = nn.Parameter(torch.zeros(input_dim))  # Bias term

    def forward(self, x):
        # Dot product between input and weight matrix (e_t = x · W)
        scores = self.attention_weights(x)  # Shape: (batch_size, input_dim)

        # Apply hyperbolic tangent activation (h_t = tanh(e_t))
        tanh_scores = torch.tanh(scores)

        # Add bias to the result (y_t = h_t + b)
        biased_scores = tanh_scores + self.bias

        # Softmax to calculate attention weights (a_t = softmax(y_t))
        attention_weights = torch.softmax(biased_scores, dim=-1)  # Shape: (batch_size, input_dim)

        # Element-wise multiplication of the input with the attention weights
        weighted_sum = x * attention_weights  # Element-wise multiplication

        return weighted_sum

#######################################################################################
## Define the CardIA-Net model
#######################################################################################
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=False):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding, bias=bias)
        self.bn = nn.BatchNorm1d(out_channels)
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x)

class InceptionModulePlus(nn.Module):
    def __init__(self, in_channels, num_kernels=32, bottleneck_channels=32):
        super(InceptionModulePlus, self).__init__()
        self.bottleneck = ConvBlock(in_channels, bottleneck_channels, kernel_size=1)
        self.convs = nn.ModuleList([
            ConvBlock(bottleneck_channels, num_kernels, kernel_size=39, padding=19),
            ConvBlock(bottleneck_channels, num_kernels, kernel_size=19, padding=9),
            ConvBlock(bottleneck_channels, num_kernels, kernel_size=9, padding=4)
        ])
        self.mp_conv = nn.Sequential(
            nn.MaxPool1d(kernel_size=3, stride=1, padding=1),
            ConvBlock(in_channels, bottleneck_channels, kernel_size=1)
        )
        self.concat = nn.Identity()
        self.norm = nn.BatchNorm1d(num_kernels * 3 + bottleneck_channels)
        self.act = nn.ReLU()
    def forward(self, x):
        x_bottleneck = self.bottleneck(x)
        conv_outputs = [conv(x_bottleneck) for conv in self.convs]
        mp_output = self.mp_conv(x)
        concatenated = torch.cat(conv_outputs + [mp_output], dim=1)
        concatenated = self.norm(concatenated)
        return self.act(concatenated)

class InceptionBlockPlus(nn.Module):
    def __init__(self, in_channels, blocks=num_incep_block, num_kernels=32, bottleneck_channels=32):
        super(InceptionBlockPlus, self).__init__()
        self.inception = nn.ModuleList([
            InceptionModulePlus(in_channels if i == 0 else num_kernels * 3 + bottleneck_channels)
            for i in range(blocks)
        ])
        self.shortcut = nn.ModuleList([
            ConvBlock(in_channels, num_kernels * 3 + bottleneck_channels, kernel_size=1),
            nn.BatchNorm1d(num_kernels * 3 + bottleneck_channels)
        ])
        self.act = nn.ModuleList([nn.ReLU(), nn.ReLU()])
        self.add = torch.add
    def forward(self, x):
        residual = x
        for module in self.shortcut:
            residual = module(residual)
        for module in self.inception:
            x = module(x)
        x = self.add(x, residual)
        for activation in self.act:
            x = activation(x)
        return x

class GAP1d(nn.Module):
    def __init__(self):
        super(GAP1d, self).__init__()
        self.gap = nn.AdaptiveAvgPool1d(output_size=1)
        self.flatten = nn.Flatten()
    def forward(self, x):
        x = self.gap(x)
        return self.flatten(x)

class LinBnDrop(nn.Module):
    def __init__(self, in_features, out_features, bias):
        super(LinBnDrop, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias)
    def forward(self, x):
        x = self.linear(x)
        return x

class InceptionTimePlus(nn.Module):
    def __init__(self):
        super(InceptionTimePlus, self).__init__()
        self.backbone = nn.Sequential(InceptionBlockPlus(in_channels=CH_No))
        self.head = nn.Sequential(GAP1d())
        self.attention = Attention(input_dim=128)
        self.final = nn.Sequential(LinBnDrop(in_features=128, out_features=3, bias=True))
    def forward(self, x):
        x = self.backbone(x)
        x = self.head(x)
        x = self.attention(x)
        return self.final(x)

#######################################################################################
# Build model, load trained parameters from working directory
#######################################################################################
model = InceptionTimePlus().to(device)
print(model)

# Load weights (no training)
state_dict = torch.load(mod_weight, map_location=device)
model.load_state_dict(state_dict)
print("Loaded trained parameters from", mod_weight)

# Loss (for reporting test loss only; class weights are not required for testing)
criterion = nn.CrossEntropyLoss()

#######################################################################################
# Testing loop
#######################################################################################
model.eval()
test_loss, correct, test_samples = 0.0, 0, 0
all_preds, all_labels = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        test_samples += labels.size(0)
        all_preds.extend(preds.view(-1).cpu().numpy())
        all_labels.extend(labels.view(-1).cpu().numpy())

print("Testing Done.")
avg_test_loss = test_loss / len(test_loader)
test_accuracy = 100.0 * correct / test_samples
print(f"Test Loss: {avg_test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Metrics (same as your code)
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:")
print(cm)

report = classification_report(all_labels, all_preds)
print("Classification Report:")
print(report)


Python Platform: Linux-4.18.0-553.58.1.el8_10.x86_64-x86_64-with-glibc2.28
PyTorch Version: 2.7.1+cu126
Using device: cuda
Expecting weight file: CardIA_Net_Trained_parameters.pth
Expecting test set   : PTB_CSN_test_set.h5
Loaded test_signals shape: (79070, 4, 300)
Loaded test_labels shape : (79070,)
Inferred CH_No from test set: 4
Test DataLoader ready.
InceptionTimePlus(
  (backbone): Sequential(
    (0): InceptionBlockPlus(
      (inception): ModuleList(
        (0): InceptionModulePlus(
          (bottleneck): ConvBlock(
            (conv): Conv1d(4, 32, kernel_size=(1,), stride=(1,), bias=False)
            (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (convs): ModuleList(
            (0): ConvBlock(
              (conv): Conv1d(32, 32, kernel_size=(39,), stride=(1,), padding=(19,), bias=False)
              (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (